# Prep

In [ ]:
%%capture
import sys; sys.path.append('../../..')

# Requires alpaca_trade_api backtrader matplotlib==3.2.2 plotly quantstats ta-lib (also requires talib C install)
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit
import backtrader as bt
import matplotlib as mpl
import pytz
import datetime
from functools import lru_cache

from src.outputs.jsonl_dump import append_jsonl

mpl.rcParams['figure.dpi'] = 250


API_KEY: str = ""
SECRET_KEY: str = ""
for l in open('../../../paper.env'):
    if l.startswith("APCA_API_KEY_ID="):
        API_KEY = l.split("=")[1].strip()
    elif l.startswith("APCA_API_SECRET_KEY="):
        SECRET_KEY = l.split("=")[1].strip()

assert API_KEY
assert SECRET_KEY

rest_api = REST(API_KEY, SECRET_KEY, 'https://paper-api.alpaca.markets')


@lru_cache()
def get_data(symbol, timeframe, start, end):
  alpaca_data = rest_api.get_bars(symbol, timeframe, start, end, adjustment='all').df
  alpaca_data.index = alpaca_data.index.tz_convert("America/New_York")
  alpaca_data = alpaca_data.between_time(datetime.time(9, 30), datetime.time(16, 0), include_end=False)
  data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol, tz=pytz.timezone('US/Eastern'))
  return data

def run_backtest(strategy, symbol: str, start: str, end: str, params, timeframe=TimeFrame.Minute, cash=10000):
  cerebro = bt.Cerebro(stdstats=True)
  cerebro.broker.setcash(cash)
  cerebro.addstrategy(strategy, **params)
  cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
  cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer')
  cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name="annualreturn")
  cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
  cerebro.adddata(get_data(symbol, timeframe, start, end))
 
  results = cerebro.run()

  cerebro.plot(iplot=True)

  strat = results[0]
  return strat

def get_roi(strategy):
  annual_returns = strategy.analyzers.annualreturn.get_analysis()
  r = 1
  for ret in annual_returns.values():
    r *= 1 + ret
  r -= 1
  return r

def get_max_drawdown(strategy):
  return strategy.analyzers.drawdown.get_analysis()['max']['drawdown']/100



# Strat

In [ ]:
import datetime

FOR_ENTRY_AND_EXIT = "FOR_ENTRY_AND_EXIT"
FOR_ENTRY = "FOR_ENTRY"
NO = "NO"

class MyStrategy(bt.Strategy):
    # TODO: over 200ema -> call, under -> put
    # TODO: day is positive -> call, day is negative -> put
    # TODO: broader market? 
    params = dict(
        use_cci = FOR_ENTRY,
        cci_period = 20,
        cci_factor = 0.015,  # default, unconfirmed
        cci_upper = 100,
        cci_lower = -100,

        use_macd = FOR_ENTRY_AND_EXIT,
        macd_settings = (5, 21, 5),

        use_emacross = FOR_ENTRY,
        emas = (9, 20, 50),
        emas_cross_criteria_any_instead_of_all = True,
        emas_any_on_cross_instead_of_while_rightly_ordered = False,

        use_psar = FOR_ENTRY,
        psar_af = 0.018,
        psar_afmax = 0.2,
        psar_period = 2,

        use_psar_lookback = FOR_ENTRY,
        psar_lookback_period = 3,

        use_mfi = NO,
        mfi_upper = 80,
        mfi_lower = 20,

        # Must be strong
        # TODO: try shorter periods
        use_adx = FOR_ENTRY,
        adx_period = 14,
        adx_min_strength = 20,

        # Must be getting stronger
        use_adx_slope = FOR_ENTRY,
        adx_slope_smoothing_period = 5,

        use_long_ema = FOR_ENTRY,
        long_ema_period = 200,

        start_of_morning = datetime.time(9, 32),
        end_of_morning = datetime.time(11, 30),
        start_of_afternoon = datetime.time(14, 30),
        end_of_day = datetime.time(15, 58),  # issues order 1m after
    )

    def add_signal(self, should_use: bool, long_signal, short_signal):
      if should_use not in (FOR_ENTRY, FOR_ENTRY_AND_EXIT):
        return
      
      if should_use == FOR_ENTRY:
        self.buy_long_signals.append(long_signal)
        self.buy_short_signals.append(short_signal)
      elif should_use == FOR_ENTRY_AND_EXIT:
        self.stay_long_signals.append(long_signal)
        self.stay_short_signals.append(short_signal)

    def __init__(self):
        self.stay_long_signals = []
        self.buy_long_signals = []

        self.stay_short_signals = []
        self.buy_short_signals = []

        if self.p.use_psar != NO:
          self.psar = bt.ind.PSAR(period=self.p.psar_period, af=self.p.psar_af, afmax=self.p.psar_afmax)
          self.add_signal(self.p.use_psar, self.psar.psar < self.data.close, self.psar.psar > self.data.close)
        
          if self.p.use_psar_lookback != NO:
            # PSAR signal should have been facing the other way 3m ago
            long_signal = self.psar.psar(-self.p.psar_lookback_period) > self.data.close(-self.p.psar_lookback_period)
            short_signal = self.psar.psar(-self.p.psar_lookback_period) < self.data.close(-self.p.psar_lookback_period)
            self.add_signal(self.p.use_psar_lookback, long_signal, short_signal)
        
        if self.p.use_adx != NO:
          self.adx = bt.ind.ADX(period=self.p.adx_period)
          self.add_signal(self.p.use_adx, self.adx > self.p.adx_min_strength, self.adx > self.p.adx_min_strength)
        
          if self.p.use_adx_slope != NO:
            self.add_signal(self.p.use_psar_lookback, self.adx.adx(0) - self.adx.adx(-1) > 0, self.adx.adx(0) - self.adx.adx(-1) > 0)
        

        if self.p.use_long_ema != NO:
          self.ema = bt.ind.EMA(period=self.p.long_ema_period)
          self.add_signal(self.p.use_long_ema, self.data.close > self.ema, self.data.close < self.ema)
 
        if self.p.use_emacross != NO:
          ema_signals = []
          for ema_period in self.p.emas:
            signal = bt.ind.EMA(period=ema_period)
            setattr(self, f'ema_{ema_period}', signal)
            ema_signals.append(signal)

          if self.p.emas_cross_criteria_any_instead_of_all:
            long_signal = bt.Max(0, 1) > 1 # False
            short_signal = bt.Max(0, 1) > 1 # False
            for i in range(len(ema_signals) - 1):
              fast, slow = ema_signals[i], ema_signals[i + 1]

              if self.p.emas_any_on_cross_instead_of_while_rightly_ordered:
                # Whenever something crosses the right way
                long_signal = bt.Or(long_signal, bt.ind.CrossUp(fast, slow))
                short_signal = bt.Or(short_signal, bt.ind.CrossDown(fast, slow))
              else:
                # ANY must be in correct order
                long_signal = bt.Or(long_signal, fast > slow)
                short_signal = bt.Or(short_signal, fast < slow)

              
          else:
            # ALL must be in correct order
            long_signal = bt.Max(0, 1) > 0 # True
            short_signal = bt.Max(0, 1) > 0 # True
            for i in range(len(ema_signals) - 1):
              fast, slow = ema_signals[i], ema_signals[i + 1]
              # fastest > fast > ... > slow > slowest
              long_signal = bt.And(long_signal, fast > slow)
              # fastest < fast < ... < slow < slowest
              short_signal = bt.And(short_signal, fast < slow)
          
          self.add_signal(self.p.use_emacross, long_signal, short_signal)
        
        if self.p.use_macd in (FOR_ENTRY, FOR_ENTRY_AND_EXIT):
          period_me1, period_me2, period_signal = self.p.macd_settings
          self.macd = bt.ind.MACD(period_me1=period_me1, period_me2=period_me2, period_signal=period_signal)
          self.add_signal(self.p.use_macd, self.macd.macd > self.macd.signal, self.macd.macd < self.macd.signal)
        
        if self.p.use_cci in (FOR_ENTRY, FOR_ENTRY_AND_EXIT):
          self.cci = bt.ind.CCI(period=self.p.cci_period, factor=self.p.cci_factor, upperband=self.p.cci_upper, lowerband=self.p.cci_lower)
          self.add_signal(self.p.use_cci, self.cci.cci > self.p.cci_upper, self.cci.cci < self.p.cci_lower)
        
        if self.p.use_mfi in (FOR_ENTRY, FOR_ENTRY_AND_EXIT):
          self.mfi = bt.talib.MFI(self.data.high, self.data.low, self.data.close, self.data.volume)
          self.add_signal(self.p.use_mfi, self.mfi > self.p.mfi_upper, self.mfi < self.p.mfi_lower)

        self.stay_long = bt.And(*self.stay_long_signals, bt.Max(0, 1) > 0)
        self.buy_long = bt.And(self.stay_long, *self.buy_long_signals)
        
        self.stay_short = bt.And(*self.stay_short_signals, bt.Max(0, 1) > 0)
        self.buy_short = bt.And(self.stay_short, *self.buy_short_signals)


    def next(self):
      now = self.data.datetime.time()

      if not (now >= self.p.start_of_morning and now <= self.p.end_of_morning) and not (now >= self.p.start_of_afternoon and now < self.p.end_of_day):
        self.order_target_size(target=0)
        return

      if self.position:
        if self.position.size > 0 and not self.stay_long:
          self.order_target_size(target=0)
        elif self.position.size < 0 and not self.stay_short:
          self.order_target_size(target=0)
      else:
        if self.buy_long:
          self.order_target_size(target=1)
        elif self.buy_short:
          self.order_target_size(target=-1)
        else:
          self.order_target_size(target=0)


# Results

In [ ]:
strategy = run_backtest(MyStrategy, "AAPL", "2022-01-01", "2022-05-01",
  {	
    'use_cci': 'FOR_ENTRY',
    'cci_period': 20,
    'cci_factor': 0.015,
    'cci_upper': -1000,
    'cci_lower': -100,
    'use_macd': 'FOR_ENTRY',
    'macd_settings': (12, 26, 9),
    'use_emacross': 'FOR_ENTRY',
    'emas': (9, 20, 50, 200),
    'emas_cross_criteria_any_instead_of_all': True,
    'emas_any_on_cross_instead_of_while_rightly_ordered': False,
    'use_psar': 'FOR_ENTRY',
    'psar_af': 0.021,
    'psar_afmax': 0.2,
    'psar_period': 2,
    'use_psar_lookback': 'FOR_ENTRY',
    'psar_lookback_period': 3,
    'use_mfi': 'NO',
    'mfi_upper': 80,
    'mfi_lower': 20,
    'use_adx': 'FOR_ENTRY',
    'adx_period': 14,
    'adx_min_strength': 20,
    'use_adx_slope': 'NO',
    'adx_slope_smoothing_period': 5,
    'use_long_ema': 'NO',
    'long_ema_period': 200,
    'start_of_morning': datetime.time(9, 32),
    'end_of_morning': datetime.time(11, 30),
    'start_of_afternoon': datetime.time(14, 30),
    'end_of_day': datetime.time(15, 58)
  }
  , cash=200, timeframe=TimeFrame(1, TimeFrameUnit.Minute)
)

In [ ]:
annual_returns = strategy.analyzers.annualreturn.get_analysis()
analysis = strategy.analyzers.tradeanalyzer.get_analysis()
# print(strategy.p.__dict__)
print(f"ROI     : {get_roi(strategy):.1%}")
print(f"Drawdown: {strategy.analyzers.drawdown.get_analysis()['max']['drawdown']/100:.1%}")
print(f"Average : {analysis['total']['total']} trades @ {analysis['pnl']['gross']['average']:.2f}")
print(f"Winners : {(analysis['won']['total'] / analysis['total']['total']):.1%} @ {analysis['won']['pnl']['average']:.2f}")
print(f"Losers  : {(analysis['lost']['total'] / analysis['total']['total']):.1%} @ {analysis['lost']['pnl']['average']:.2f}")

In [ ]:
import quantstats
portfolio_stats = strategy.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)
quantstats.reports.basic(returns)

In [ ]:
import src.results.from_backtest as from_backtest

import src.results.metadata as metadata
import src.types as types

transactions.index = transactions.index.tz_convert("America/New_York")
transactions['time'] = transactions.index

from_backtest.write_results("Apples", [types.FilledOrder(intention=None, symbol=t['symbol'], quantity=t['amount'], price=t['price'], datetime=t['time'].to_pydatetime()) for t in transactions.to_dict('records')], metadata.Metadata(commit_id="", last_updated=datetime.datetime.now()))

# Optimize

In [ ]:
gridsearch = dict(
    use_cci = [FOR_ENTRY], # NO = both upper and lower ignore
    cci_period = [20],
    cci_factor = [0.015],
    cci_upper = [-1000],
    cci_lower = [-100],

    use_macd = [FOR_ENTRY],
    macd_settings = [
      # (5, 21, 5),
      # (9, 21, 5),
      # (9, 21, 8),
      (12, 26, 9),
    ],

    use_emacross = [FOR_ENTRY],
    emas = [
      # (9, 20, 50),
      (9, 20, 50, 200),
    ],
    emas_cross_criteria_any_instead_of_all = [True],
    emas_any_on_cross_instead_of_while_rightly_ordered = [False],

    use_psar = [FOR_ENTRY],
    psar_af = [0.021],
    psar_afmax = [0.2],
    psar_period = [2],

    use_psar_lookback = [FOR_ENTRY],
    psar_lookback_period = [3],  # 3 over 5, 7, 9

    use_mfi = [NO], # NO = both upper and lower ignore
    mfi_upper = [80], # 0=ignore
    mfi_lower = [20], # 100=ignore

    use_adx = [FOR_ENTRY],
    adx_period = [14],
    adx_min_strength = [20],

    # Must be getting stronger
    use_adx_slope = [NO],
    adx_slope_smoothing_period = [5],

    # TODO: do 200ema on D chart (not 1m)
    use_long_ema = [NO],
    long_ema_period = [200],

    start_of_morning =   [datetime.time( 9, 32)],
    end_of_morning =     [datetime.time(11, 30)],  # 11:30 over 11, 10:30, and 15:58
    start_of_afternoon = [datetime.time(14, 30)],
    end_of_day =         [datetime.time(15, 58)],  # issues order 1m after
)

time_per = datetime.timedelta(seconds=14)

choices = 1
for l in gridsearch.values():
  choices *= len(l)

estimated_time = time_per * choices
print(f"Estimated time (@ {time_per}, {choices}): {estimated_time}")
start_time = datetime.datetime.now()
print(f"(ending at {start_time + (time_per * choices)})")
print("=" * 80)

cerebro = bt.Cerebro(stdstats=True, maxcpus=1)  # inside of VS Code editor, this does not go well
cerebro.broker.setcash(200)
cerebro.optstrategy(MyStrategy, **gridsearch)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer')
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name="annualreturn")
cerebro.adddata(get_data("AAPL", TimeFrame(1, TimeFrameUnit.Minute), "2022-01-01", "2022-05-01"))

runs = cerebro.run()
backtest_results = [results[0] for results in runs]



end_time = datetime.datetime.now()
print(f"Done. Took {end_time - start_time}")
print(f"time per: {(end_time - start_time) / choices}")

for strategy in list(sorted(backtest_results, key=get_roi, reverse=True))[:10]:
  analysis = strategy.analyzers.tradeanalyzer.get_analysis()
  if analysis["total"]["total"] == 0:
    continue
  annual_returns = strategy.analyzers.annualreturn.get_analysis()
  print(strategy.p.__dict__)
  print(f"ROI     : {get_roi(strategy):.1%}")
  print(f"Drawdown: {get_max_drawdown(strategy):.1%}")
  print(f"Average : {analysis['total']['total']} trades @ {analysis['pnl']['gross']['average']:.2f}")
  print(f"Winners : {(analysis['won']['total'] / analysis['total']['total']):.1%} @ {analysis['won']['pnl']['average']:.2f}")
  print(f"Losers  : {(analysis['lost']['total'] / analysis['total']['total']):.1%} @ {analysis['lost']['pnl']['average']:.2f}")
  print()